In [205]:
import os
import sys
os.chdir('/home/ajinkya/Guided_HLM/HeapStatee/')
# sys.path.append('HeapModel')
from copy import deepcopy
import angr
from angr import SimStatePlugin, SimProcedure
from HeapModel.mstate import HeapState

class HeapPlugin(SimStatePlugin):
    def __init__(self, h=None):
        super().__init__()
        if(h is None):
            self.heap_state = HeapState(0)
        else:
            self.heap_state = h

    def set_heap(self, h):
        self.heap_state = h

    @SimStatePlugin.memo
    def copy(self, memo):
        heap_copy = deepcopy(self.heap_state)
        return HeapPlugin(heap_copy)

b = angr.Project('AngrState/TestCases/UAF')
ss = b.factory.blank_state()

ss.register_plugin('my_heap', HeapPlugin())

class Malloc(SimProcedure):
    i = 0
    def run(self, size):
        print(f'malloc called {self.i}')
        print(size)

        self.i+=1

class Free(SimProcedure):
    i = 0
    def run(self, address):
        print(f'free called {self.i}')
        print(address)
        self.i+=1

b.hook_symbol('malloc', Malloc())
b.hook_symbol('free', Free())



sm = b.factory.simulation_manager(ss)


In [234]:
sm.step()


malloc called 0
<SAO <BV64 0x10>>


<SimulationManager with 1 active>